In [68]:
from qiskit import *
%matplotlib inline
from qiskit.tools.visualization import plot_histogram

In [69]:
secretnumber = '101010101110010100010100111'
n =  len(secretnumber)

In [70]:
circuit = QuantumCircuit(n+1, n)

#circuit.h([0,1,2,3,4,5])
circuit.h(range(n))
circuit.x(n)
circuit.h(n)
circuit.barrier()

for i,yesno in enumerate(reversed(secretnumber)):
    if (yesno == '1'):
        circuit.cx(i,n)

circuit.barrier()
circuit.h(range(n))
circuit.measure(range(n),range(n))

In [71]:
simulator = Aer.get_backend('qasm_simulator')
result = execute(circuit, backend = simulator, shots = 1).result()
counts = result.get_counts()
print(counts)